# 동적 페이지 크롤링하기(3)

# 네이버 웹툰 댓글 가져오기

해당 웹툰의 해당 화 페이지로 requests.get 했더니,  
안가져와지네?? -> iframe  
iframe_url로 requests.get했더니, 
안가져와지네?? -> javascript  
network tab에서 preview 보면서 requests_url을 찾아낸다.

### 1. 하나의 웹툰, 하나의 화 베스트 댓글 가져오기

In [ ]:
import json 
import requests
from tqdm.notebook import tqdm

In [ ]:
webtoon = {
        'index' : [],
        '댓글' : [],
        '공감수' : [],
        '비공감수' :[]
        }

In [ ]:
url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_cv=20221018144728&_callback=jQuery112409767973216970143_1667194741406&lang=ko&country=KR&objectId=783053_52&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&pageType=default&page=1&initialize=true&userType=&useAltSort=true&replyPageSize=10&_=1667194741408'

headers = {
        'referer': 'https://comic.naver.com/comment/comment?titleId=783053&no=52',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
}

resp = requests.get(url, headers=headers)
result = resp.text

In [ ]:
result = result.replace('jQuery112409767973216970143_1667194741406(', '')[:-2]
result = json.loads(result)
result


{'success': True,
 'code': '1000',
 'message': '요청을 성공적으로 처리하였습니다.',
 'lang': 'ko',
 'country': 'KR',
 'result': {'commentList': [{'ticket': 'comic',
    'objectId': '783053_52',
    'categoryId': '*',
    'templateId': 'default_v2',
    'commentNo': '444588718',
    'parentCommentNo': '444588718',
    'replyLevel': 1,
    'replyCount': 306,
    'replyAllCount': 334,
    'replyPreviewNo': None,
    'replyList': None,
    'imageCount': 0,
    'imageList': None,
    'imagePathList': None,
    'imageWidthList': None,
    'imageHeightList': None,
    'commentType': 'txt',
    'stickerId': None,
    'sticker': None,
    'sortValue': 1663595880447,
    'contents': '정답: 교육부!',
    'userIdNo': 'ey99F',
    'exposedUserIp': None,
    'lang': 'ko',
    'country': 'KR',
    'idType': 'naver',
    'idProvider': 'naver',
    'userName': '082',
    'userProfileImage': 'https://profile-phinf.pstatic.net/33344/229b0d5f34a408cfe79790c9fcadb1ad78b13f71570d482c4ce3d12f7d4fbf03.jpg',
    'profileType': 'n

In [ ]:
#'result' 'commentList'  [] 'contents'
                            # 'sympathyCount': 13976,
                            # 'antipathyCount': 136,
contents, syms, antis = [], [], []

commentList = result['result']['commentList']

for comment in commentList:
    contents.append(comment['contents'])
    syms.append(comment['sympathyCount'])
    antis.append(comment['antipathyCount'])
contents, syms, antis

(['정답: 교육부!',
  '포골 한칸 더 ?',
  '자유를 억압하고 본인들의 바람만을 원하고 강요하는 학부모들을 말하는듯',
  '뭐긴 뭐야 롤이지\n\n내가 오늘도 몇번을 죽었는데',
  '정답 : 한라인만 더',
  '"사람들에게 잊혀질때"',
  '정답 : 눈나\n헤으응 눈나 나 죽어',
  'ㅋㅋㅋ차에서 지루하다고 아저씨랑 넌센스퀴즈놀이하노',
  '펜ㅌ닐',
  '정답. 카페맘클럽',
  '청소년 자살율 자살원인 말하는듯',
  '정답: 킥보드 이바리 삼바리 노뚝에 거의뭐 죽인게 아니라 자살이지',
  '정답 : 눈나\n눈나 나 죽어',
  '정답이 교육부라고? 서태지 교실 이데아 가 아니고?',
  '자살 말하는건가'],
 [13976,
  8710,
  3373,
  3351,
  1700,
  1606,
  1062,
  660,
  635,
  519,
  460,
  420,
  401,
  399,
  389],
 [136, 55, 56, 48, 28, 25, 51, 16, 34, 24, 18, 18, 33, 43, 17])

### 2. 하나의 웹툰, 하나의 화 전체 댓글 가져오기

In [ ]:
import json 
import requests
from tqdm.notebook import tqdm

In [ ]:
# 전체 댓글의 맨 뒤페이지 번호 가져오기
url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'

params ={
        'ticket' : 'comic',  
        'templateId' : 'webtoon',  
        'pool' : 'cbox3',  
        '_callback' : '', # jQuery112409091552467251651_1615854232166 -> ''  
        'lang' : 'ko',  
        'country' : 'KR',  
        'objectId' : '783053_52', # 웹툰의 고유 아이디_해당화 인덱스  
        'pageSize' : 15,  
        'indexSize' : 10,  
        'listType' : 'OBJECT',  
        'pageType' : 'default',  
        'page' : 1, # 전체댓글의 페이지 번호 
        'refresh' : 'true',  
        'sort' : 'NEW' # 전체댓글 URL의 고유 쿼리 
        }

headers = {
        'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=650305&no=298',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'

}

resp = requests.get(url, params=params, headers=headers)
 
resp

<Response [200]>

In [ ]:
resp.text # _callback({ ~~~~~ }); -> { ~~~~~ }만 뽑아내야한다. 

'_callback({"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"783053_52","categoryId":"*","templateId":"default_v2","commentNo":"446459302","parentCommentNo":"446459302","replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1667199748212,"contents":"???:모기 아니냐","userIdNo":"22T9r","exposedUserIp":null,"lang":"ko","country":"KR","idType":"naver","idProvider":"naver","userName":"선영이","userProfileImage":"https://profile-phinf.pstatic.net/05414/7ce5711dc4829e67368ca9990d1c23876e61ab73bbc44d776f43c004c6db95d1.jpg","profileType":"naver","modTime":"2022-10-31T16:02:28+0900","modTimeGmt":"2022-10-31T07:02:28+0000","regTime":"2022-10-31T16:02:28+0900","regTimeGmt":"2022-10-31T07:02:28+0000","sympathyCount":0,"anti

In [ ]:
resp.text[10:-2]

'{"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"783053_52","categoryId":"*","templateId":"default_v2","commentNo":"446459302","parentCommentNo":"446459302","replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1667199748212,"contents":"???:모기 아니냐","userIdNo":"22T9r","exposedUserIp":null,"lang":"ko","country":"KR","idType":"naver","idProvider":"naver","userName":"선영이","userProfileImage":"https://profile-phinf.pstatic.net/05414/7ce5711dc4829e67368ca9990d1c23876e61ab73bbc44d776f43c004c6db95d1.jpg","profileType":"naver","modTime":"2022-10-31T16:02:28+0900","modTimeGmt":"2022-10-31T07:02:28+0000","regTime":"2022-10-31T16:02:28+0900","regTimeGmt":"2022-10-31T07:02:28+0000","sympathyCount":0,"antipathyCount

In [ ]:
result = json.loads(resp.text[10:-2]) # resp.text[10:-2] -> { ~~~~~ } 
result

{'success': True,
 'code': '1000',
 'message': '요청을 성공적으로 처리하였습니다.',
 'lang': 'ko',
 'country': 'KR',
 'result': {'commentList': [{'ticket': 'comic',
    'objectId': '783053_52',
    'categoryId': '*',
    'templateId': 'default_v2',
    'commentNo': '446459302',
    'parentCommentNo': '446459302',
    'replyLevel': 1,
    'replyCount': 0,
    'replyAllCount': 0,
    'replyPreviewNo': None,
    'replyList': None,
    'imageCount': 0,
    'imageList': None,
    'imagePathList': None,
    'imageWidthList': None,
    'imageHeightList': None,
    'commentType': 'txt',
    'stickerId': None,
    'sticker': None,
    'sortValue': 1667199748212,
    'contents': '???:모기 아니냐',
    'userIdNo': '22T9r',
    'exposedUserIp': None,
    'lang': 'ko',
    'country': 'KR',
    'idType': 'naver',
    'idProvider': 'naver',
    'userName': '선영이',
    'userProfileImage': 'https://profile-phinf.pstatic.net/05414/7ce5711dc4829e67368ca9990d1c23876e61ab73bbc44d776f43c004c6db95d1.jpg',
    'profileType': 'nav

In [ ]:
totalPages = result['result']['pageModel']['totalPages']
totalPages

93

In [ ]:
# best 댓글
# url  :  'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery112408122866196452874_1612750131447&lang=ko&country=KR&objectId=650305_298&pageSize=15&indexSize=10&listType=OBJECT&pageType=default&page=1&initialize=true&useAltSort=true&replyPageSize=10&_=1612750131448'

# 전체 댓글
url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'

for page in tqdm(range(1, totalPages+1)):
    params ={
            'ticket' : 'comic',  
            'templateId' : 'webtoon',  
            'pool' : 'cbox3',  
            '_callback' : '',  
            'lang' : 'ko',  
            'country' : 'KR',  
            'objectId' : '650305_298', # 웹툰의 고유 아이디_해당화 인덱스  
            'pageSize' : 15,  
            'indexSize' : 10,  
            'listType' : 'OBJECT',  
            'pageType' : 'default',  
            'page' : page, # 전체댓글의 페이지 번호 
            'refresh' : 'true',  
            'sort' : 'NEW', # 전체댓글 URL의 고유 쿼리 
            '_' : 1612753212403
            }

    headers = {
        'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=650305&no=298',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
    }

    resp = requests.get(url, params=params, headers=headers)




    result = json.loads(resp.text[10:-2])
    result
    # result commentList : [] 'antipathyCount', 'contents', 'commentNo', 'sympathyCount'

    # webtoon = {
    #         'index' : [],
    #         '댓글' : [],
    #         '공감수' : [],
    #         '비공감수' :[]
    #         }

    commentList = result['result']['commentList']

    for comment in commentList:
        webtoon['index'].append(comment['commentNo'])
        webtoon['댓글'].append(comment['contents'])
        webtoon['공감수'].append(comment['sympathyCount'])
        webtoon['비공감수'].append(comment['antipathyCount'])


  0%|          | 0/93 [00:00<?, ?it/s]

In [ ]:
df_webtoon = pd.DataFrame(webtoon)
df_webtoon

NameError: ignored

In [ ]:
df_webtoon.info()

In [ ]:
df_webtoon[df_webtoon['공감수'] >= 100]

,index,댓글,공감수,비공감수
2095,407452914,"하,....무케 대사 넘 눈물난다 저 어린애가...뭘알겠어\n그리고 추이도....그...",3411,49
2098,407452764,추이가 무케 볼때 염주는 일부러 푼건가...아님 그냥 원래 모습으로 표현된걸까 참 ...,2649,47
2201,407450943,이번화 요약\n1기억찾은 추이형님이 폐하 구다국으로 데려다드림\n2가는 길에 무케 ...,4926,101
2235,407450375,폐하를 추이가 데리고 온거잖아요...데려다 놓고 무케랑 작별인사하고 이제 붉은 산으...,2674,61
2236,407450364,추이 말하지않아도 너의 감정들이 다보인다ㅠㅠㅠㅠㅠㅠㅠ 진짜 우째.......,2905,50
2238,407450316,"‘아저씨한테 아빠냄새 난다’고 하고, 나중엔 ‘아빠한테 아저씨 냄새 난다’고 하네..ㅠㅜ",5505,53
2243,407450260,매 화마다 감동적이고 격동적이길 바라지 마라 \n작가님 마음가는대로 그릴 수 있도록...,3475,204
2244,407450259,무케한테 팔을 내주고 있는 추이는 꿈속 상상이 아니라 진짜 추이구나. 무커가 한 대...,4625,39
2286,407449707,"중간에 시 미친거아니냐ㅋㅋㅋ \n너희 대장""님""을 이령""놈""이 데리고 있다네ㅋㅋ",10459,76
2302,407449473,와 추이 깨어나고 구다국왕 모셔다놓고 무케 한번 보고 바로 붉은 산 직행했구나,11444,45


### 3. 전체 웹툰의 최신화의 베스트 댓글 가져오기

In [ ]:
import json 
import requests
from tqdm.notebook import tqdm

#### 3-1) 모든 웹툰의 정보 가져오기
- 제목
- 링크
- title_id
- 최신화 no

In [ ]:
tits, urls, ids, nos = [], [], [], [] 

In [ ]:
url = 'https://comic.naver.com/webtoon/weekday'
resp = requests.get(url)

soup = BeautifulSoup(resp.content, 'lxml')
a_tags = soup.select('div.list_area.daily_all a.title')

for a_tag in tqdm(a_tags):
    tits.append(a_tag['title'])
    urls.append('https://comic.naver.com'+a_tag['href'])

    id = a_tag['href'].split('titleId=')[-1].split('&')[0]
    # print(id)
    # break
    ids.append(id)

for url in tqdm(urls):
    resp = requests.get(url)
    # print(resp.url)

    soup = BeautifulSoup(resp.content, 'lxml')
    last_no = int(soup.select('td.title a')[0]['href'].split('&')[1].strip('no='))
    # print(last_no)
    # break
    nos.append(last_no)

  0%|          | 0/565 [00:00<?, ?it/s]

  0%|          | 0/565 [00:00<?, ?it/s]

[103,
 358,
 54,
 181,
 412,
 5,
 55,
 23,
 18,
 43,
 127,
 340,
 87,
 147,
 60,
 7,
 20,
 846,
 65,
 100,
 67,
 2,
 88,
 25,
 8,
 64,
 139,
 62,
 66,
 8,
 87,
 1,
 59,
 51,
 2,
 309,
 18,
 35,
 48,
 37,
 16,
 21,
 48,
 75,
 52,
 29,
 49,
 46,
 14,
 92,
 29,
 17,
 34,
 25,
 77,
 119,
 39,
 8,
 35,
 51,
 69,
 25,
 19,
 43,
 37,
 29,
 26,
 45,
 53,
 24,
 48,
 83,
 25,
 33,
 38,
 91,
 21,
 53,
 71,
 54,
 236,
 21,
 57,
 40,
 133,
 141,
 73,
 432,
 186,
 46,
 60,
 185,
 256,
 54,
 14,
 91,
 22,
 7,
 149,
 14,
 274,
 880,
 64,
 83,
 47,
 134,
 40,
 14,
 11,
 144,
 48,
 15,
 111,
 405,
 343,
 68,
 24,
 105,
 77,
 46,
 36,
 98,
 34,
 315,
 17,
 85,
 92,
 41,
 21,
 34,
 27,
 53,
 38,
 27,
 7,
 5,
 17,
 79,
 52,
 71,
 15,
 46,
 74,
 68,
 77,
 15,
 64,
 82,
 77,
 22,
 81,
 38,
 17,
 82,
 25,
 1,
 1,
 52,
 126,
 326,
 87,
 200,
 6,
 213,
 35,
 101,
 99,
 117,
 131,
 2,
 47,
 13,
 57,
 350,
 99,
 108,
 21,
 113,
 98,
 37,
 168,
 201,
 18,
 19,
 39,
 17,
 26,
 172,
 180,
 355,
 8,
 5,
 1,
 342,
 10

In [ ]:
toon_info = {}

for i in range(len(tits)):
    toon_info[tits[i]] = {'title_id' : ids[i], 'no' : nos[i]}
toon_info

{'참교육': {'title_id': '758037', 'no': 103},
 '뷰티풀 군바리': {'title_id': '648419', 'no': 358},
 '퀘스트지상주의': {'title_id': '783052', 'no': 54},
 '장씨세가 호위무사': {'title_id': '728750', 'no': 181},
 '윈드브레이커': {'title_id': '602910', 'no': 412},
 '퍼니게임': {'title_id': '801035', 'no': 5},
 '팔이피플': {'title_id': '774863', 'no': 55},
 '신화급 귀속 아이템을 손에 넣었다': {'title_id': '795297', 'no': 23},
 '버림받은 왕녀의 은밀한 침실': {'title_id': '796867', 'no': 18},
 '호랑신랑뎐': {'title_id': '795643', 'no': 43},
 '앵무살수': {'title_id': '739115', 'no': 127},
 '소녀의 세계': {'title_id': '654774', 'no': 340},
 '잔불의 기사': {'title_id': '768536', 'no': 87},
 '백수세끼': {'title_id': '733074', 'no': 147},
 '똑 닮은 딸': {'title_id': '774866', 'no': 60},
 '사이다걸': {'title_id': '800333', 'no': 7},
 '절대검감': {'title_id': '796075', 'no': 20},
 '히어로메이커': {'title_id': '21815', 'no': 846},
 '물어보는 사이': {'title_id': '772725', 'no': 65},
 '리턴 투 플레이어': {'title_id': '752414', 'no': 100},
 '꼬리잡기': {'title_id': '758666', 'no': 67},
 '인섹터': {'title_id': '801710', 'no': 

#### 3-2) 수집한 웹툰 정보를 이용하여, 전체 웹툰의 댓글가져오기
- 최신화
- 베스트 댓글

In [ ]:
reviews = {
    '댓글' : [],
    '공감수' : [],
    '비공감수' : [], 
    '제목' : []
    }

for title, info in tqdm(toon_info.items()):

    print(title)

    title_id = info['title_id']
    no = info['no']

    url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'


    params ={
            'ticket' : 'comic',  
            'templateId' : 'webtoon',  
            'pool' : 'cbox3',  
            '_callback' : '', # jQuery112409091552467251651_1615854232166 -> ''  
            'lang' : 'ko',  
            'country' : 'KR',  
            'objectId' : f'{title_id}_{no}', # 웹툰의 고유 아이디_해당화 인덱스  
            'pageSize' : 15,  
            'indexSize' : 10,  
            'listType' : 'OBJECT',  
            'pageType' : 'default',  
            'page' : 1, # 전체댓글의 페이지 번호 
            'refresh' : 'true',  
            }

    headers = {
            'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=650305&no=298',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
            }

    resp = requests.get(url, params=params, headers=headers)

    jsn = resp.text[10:-2]

    dict_resp = json.loads(jsn)
    dict_resp 

    commentList = dict_resp['result']['commentList']
    # len(commentList)

    for comment in commentList:
        reviews['댓글'].append(comment['contents'])
        reviews['공감수'].append(comment['sympathyCount'])
        reviews['비공감수'].append(comment['antipathyCount'])
        reviews['제목'].append(title)

  0%|          | 0/550 [00:00<?, ?it/s]

참교육
뷰티풀 군바리
퀘스트지상주의
장씨세가 호위무사
윈드브레이커
퍼니게임
팔이피플
신화급 귀속 아이템을 손에 넣었다
버림받은 왕녀의 은밀한 침실
호랑신랑뎐
앵무살수
소녀의 세계
잔불의 기사
백수세끼
똑 닮은 딸
사이다걸
절대검감
히어로메이커
물어보는 사이
리턴 투 플레이어
꼬리잡기
인섹터
순정말고 순종
북부 공작님을 유혹하겠습니다
헬크래프트
더블클릭
결혼생활 그림일기
또다시, 계약 부부
아, 쫌 참으세요 영주님!
도깨비 부른다
황제와의 하룻밤
랭커
오빠집이 비어서
제왕: 빛과 그림자
퇴근 후에 만나요
꿈의 기업
불청객
파운더
이제야 연애
미니어처 생활백서
아슈타르테
우산 없는 애
하루의 하루
수영만화일기
신군
다시쓰는 연애사
원작은 완결난 지 한참 됐습니다만
레지나레나 - 용서받지 못한 그대에게
굿바이 유교보이
그림자 신부
오늘의 비너스
메리의 불타는 행복회로
입술이 예쁜 남자
버그이터
최후의 금빛아이
백호랑
파견체
행운을 부탁해!
아마도
달로 만든 아이
모스크바의 여명
루크 비셸 따라잡기
세번째 로망스
나만의 고막남친
매지컬 급식:암살법사
경비실에서 안내방송 드립니다
지옥연애환담
악녀 18세 공략기
역주행!
모노마니아
슈퍼스타 천대리
사막에 핀 달
흔들리는 세계로부터
결혼공략
디나운스
오로지 오로라
별을 쫓는 소년들
김부장
대학원 탈출일지
내가 키운 S급들
여신강림
마루는 강쥐
1을 줄게
멸망 이후의 세계
하루만 네가 되고 싶어
중증외상센터 : 골든 아워
용사가 돌아왔다
삼국지톡
헬58
천마는 평범하게 살 수 없다
악몽의 형상
집이 없어
랜덤채팅의 그녀!
하북팽가 막내아들
시체기사 군터
호랑이 들어와요
초인의 게임
사표내고 이계에서 힐링합니다
올가미
웅크
원주민 공포만화
윌유메리미
덴큐
나타나주세요!
애옹식당
용왕님의 셰프가 되었습니다
늑대처럼 홀로
왕게임
헥토파스칼
견우와 선녀
택배 왔습니다
로잘린 보가트
빅맨
에이머
은주의 방 2~3부
빌런투킬
사공은주
플레이, 플리
아이즈
또 다른 사랑
내남친 킹카만들기
나의 플랏메이트
벽간소음
제로게임
여우자매
풋내

In [ ]:
toon_all_rev = pd.DataFrame(reviews)
toon_all_rev

,댓글,공감수,비공감수,제목
0,저 비키니가 되고싶어요 저 비키니가 되고싶어요 저 비키니가 되고싶어요 저 비키니가 ...,8720,698,참교육
1,아니...비키니를 입힐거면 갖다대지만 말고 제대로 입혀야죠....\n\n바다. 기대...,6578,313,참교육
2,주님 부처님 알라신님 오늘 좀 많이 갈거 같습니다,5806,42,참교육
3,신 등장,4591,22,참교육
4,저 학생의 입장에선\n임한림이 신이다.\n\n앞으로의 인생에 계속 기억되고 고마운 ...,2808,17,참교육
...,...,...,...,...
5384,아 설마 저번에 기도하던게 산호쪽이 아니라 준영이가 죽길 기도하던거였나,569,3,"구해줘, 호구!"
5385,아 할배 나가라고요ㅠㅠ..!!,286,1,"구해줘, 호구!"
5386,아 그냥 둘이 행복하게해주세요ㅠㅠ,173,0,"구해줘, 호구!"
5387,자기 손자 살리겠다고 준영이 이용한겨?ㄷㄷ,127,0,"구해줘, 호구!"


### 4. 원하는 웹툰을 입력받아, 해당 웹툰의 전체 댓글 가져오는 함수 만들기
- 함수화하기

#### 4-1) 모든 웹툰의 정보 수집하는 함수 만들기
- 입력인자 : X
- 출력인자 : toon_info

In [ ]:
def scraping_total_toon_info():
    tits, urls, ids, nos = [], [], [], [] 
    toon_info = {} # 전체 웹툰 정보 저장 딕셔너리

    url = 'https://comic.naver.com/webtoon/weekday'
    resp = requests.get(url)

    soup = BeautifulSoup(resp.content, 'lxml')
    a_tags = soup.select('div.list_area.daily_all a.title')

    for a_tag in tqdm(a_tags):
        tits.append(a_tag['title'])
        urls.append('https://comic.naver.com'+a_tag['href'])

        id = a_tag['href'].split('titleId=')[-1].split('&')[0]
        # print(id)
        # break
        ids.append(id)

    for url in tqdm(urls):
        resp = requests.get(url)
        # print(resp.url)

        soup = BeautifulSoup(resp.content, 'lxml')
        last_no = int(soup.select('td.title a')[0]['href'].split('&')[1].strip('no='))
        # print(last_no)
        # break
        nos.append(last_no)

    for i in range(len(tits)):
        toon_info[tits[i]] = {'title_id' : ids[i], 'no' : nos[i]}
    return toon_info

In [ ]:
scraping_total_toon_info()

#### 4-2) 원하는 웹툰의 모든 댓글 크롤링하는 함수 만들기 
- 입력파라미터 : 웹툰이름
- return 값 : reviews

In [ ]:
def toon_review_scraping(mytoon):
    title_id = toon_info[mytoon]['title_id']
    last_no = toon_info[mytoon]['no']
    
    reviews = {
        '댓글' : [],
        '공감수' : [],
        '비공감수' : [],
        '화' : []
    }

    for no in tqdm(range(1, last_no+1)):
        url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'


        params ={
                'ticket' : 'comic',  
                'templateId' : 'webtoon',  
                'pool' : 'cbox3',  
                '_callback' : 'jQuery11240590927015056502_1658103118387', # jQuery112409091552467251651_1615854232166 -> ''  
                'lang' : 'ko',  
                'country' : 'KR',  
                'objectId' : '{}_{}'.format(title_id, no), # 웹툰의 고유 아이디_해당화 인덱스  
                'pageSize' : 15,  
                'indexSize' : 10,  
                'listType' : 'OBJECT',  
                'pageType' : 'default',  
                'page' : 1, # 전체댓글의 페이지 번호 
                'refresh' : 'true',  
                }

        headers = {
                'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=650305&no=298',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
        }

        resp = requests.get(url, params=params, headers=headers)
        
        jsn = resp.text[41:-2]

        dict_resp = json.loads(jsn)

        commentList = dict_resp['result']['commentList']
        # len(commentList)

        for comment in commentList:
            reviews['댓글'].append(comment['contents'])
            reviews['공감수'].append(comment['sympathyCount'])
            reviews['비공감수'].append(comment['antipathyCount'])
            reviews['화'].append(no)
    return reviews

In [ ]:
mytoon = input('웹툰제목을 입력해주세요:')

toon_review_scraping(mytoon)